In [1]:
#########################################################
# Developed By: Sebastian Maurice, PhD
########################################################

import maadsbml
import json
import os
import time
# Uncomment IF using jupyter notebook
import nest_asyncio
# Uncomment IF using jupyter notebook
nest_asyncio.apply()

In [2]:
host='http://127.0.0.1'
port=5595
######################### Change these two folder to your local paths that you used for the volume mappings in Docker
########### Local Paths on Linux/Mac
localstagingfolder = "/Users/admin/maads/staging" # change this folder to your local mapped staging folder
localexceptionfolder = "/Users/admin/maads/exception" # change this folder to your local mapped exception folder

########### Local Paths on Windows
#localstagingfolder = "c:\\maads\\agentfilesdocker\\dist\\staging" # change this folder to your local mapped staging folder
#localexceptionfolder = "c:\\maads\\agentfilesdocker\\dist\\maadsweb\\exception" # change this folder to your local mapped exception folder

#########################################################

def readifbrokenpipe(jres,hasseasonality):
      # this function is called if there is a broken pipe network issue
      pkey=""
      algofile=""        
      jsonalgostr = ""
    
      pkey= jres.get('AlgoKey')
    
      maadsbmlfile="%s/%s.txt.working" % (localstagingfolder,pkey)
      if hasseasonality == 1:
        algojsonfile="%s/%s_trained_algo_seasons.json" % (localexceptionfolder,pkey)
      else:
        algojsonfile="%s/%s_trained_algo_no_seasons.json" % (localexceptionfolder,pkey)
      
      while True:
          time.sleep(5)            
          if os.path.isfile(maadsbmlfile): 
               continue
          elif os.path.isfile(algojsonfile):
                # Read the json            
              with open(algojsonfile) as f:
                  jsonalgostr = json.load(f)
              break # maadsbml finished
      return jsonalgostr

def hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality,deepanalysis):
  #def hypertraining(host,port,filename,dependentvariable,removeoutliers=0,hasseasonality=0,
    #summer='6,7,8',winter='11,12,1,2',shoulder='3,4,5,9,10',trainingpercentage=70,shuffle=0,
    #deepanalysis=0,username='admin',timeout=1200,company='otics',password='123',email='support@otics.ca',
    #usereverseproxy=0,microserviceid='',maadstoken='123',mode=0):


  #host,port,
  #filename= raw data file in csv format - Note this file is stored on your host machine the DOCKER container needs to be mapped to this volume using -v
  #dependentvariable= dependent variable name - this is the column name in the csv file
  # the file should have a Date column in the format Month/Day/Year
  #username= you can specify a username
  #mode=0
  #timeout=180 - you can modify this in seconds if your data file is large
  #company= change this to the name of your company
  #removeoutliers= specify 1 or 0, 1=remove outliers, 0 do not remove outliers,
  #hasseasonality= specify 1 or 0 to indicate date is affected by seasonaility - 1 = seasonality, 0 = no seasonality,
  #summer= specify the summer months ie. '6,7,8', or set to -1 for no summer
  #winter= specify winter months i.e. '11,12,1,2', or -1 for no winter
  #shoulder= specify shoulder months i.e. '3,4,5,9,10', or -1 for no shoulder season
  #trainingpercentage= specify training percentage i.e. 70, the value represents a percentage to split training and test
  #shuffle= specify 1 or 0 to shuffle the data, 1= shuffle, 0 = no shuffle
  #deepanalysis= specify 1 or 0, 1=deepanalysis, note this will run through deeper algorithms but will take longer, 0 = no deep analysis, this will
  #password='123', - leave as is
  #email='support@otics.ca', - leave as is
  #usereverseproxy=0, - leave as is
  #microserviceid='', leave as is
  #maadstoken='123' leave as is
  summer='6,7,8'
  winter='11,12,1,2'
  shoulder='3,4,5,9,10'
  #shoulder='-1'
  trainingpercentage=75
  shuffle=1
  res=maadsbml.hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality,summer,winter,shoulder,trainingpercentage,shuffle,deepanalysis)
  jres = json.loads(res)

  if jres.get('BrokenPipe') != None: # check if the hypertraining function experienced a brokenpipe - if so wait 
        try:
          res=readifbrokenpipe(jres,hasseasonality)
        except Exception as e:
          print(e)  
           
  print(res)


def hyperprediction(pkey,host,port,inputdata,username):
  
  res=maadsbml.hyperpredictions(pkey,inputdata,host,port,username)
  print(res)

def hyperpredictioncustom(pkey,host,port,inputdata,username,algoname,season):
  res=maadsbml.hyperpredictions(pkey,inputdata,host,port,username,algoname,season)
  print(res)
  
def algoinfo(pk):
   res=maadsbml.algodescription(host,port,pk)
   print(res)

def rundemo(demotype):
   global host
   global port
    # if demotype=1 then Regression will be run
    # if demotype=0 then Classification will be run
   res=maadsbml.rundemo(host,port,demotype)
   print(res)

def abort(host,port):
   res=maadsbml.abort(host,port)
   print(res)



In [ ]:
# ############Function Commands
# Algoinfo
pk='admin_aesopowerdemand_csv'
algoinfo(pk)

#pk='admin_aesopowerdemandlogistic_csv'
#algoinfo(pk)

# ############Abort
#abort(host,10000)

# ############Rundemo
rundemo(0)


http://127.0.0.1:5595/?algoinfo=1&pkey=admin_aesopowerdemand_csv
{"RUN DATE": "2024/04/10", "RUN TIME": "1500", "ALGO KEY": "admin_aesopowerdemand_csv", "ALGO NAME": "TheilSenRegressor", "ALGO DESCRIPTION": "Theil-Sen Estimator (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.TheilSenRegressor.html#sklearn.linear_model.TheilSenRegressor) - Theil-Sen Estimator: robust multivariate regression model.", "FORECAST ACCURACY": 0.59, "TRAINING SAMPLE SIZE": 897, "NUMBER OF INPUTS": 3, "SEASON": "N", "DEPENDENT VARIABLE": "AESO_Power_Demand", "TOTAL NUMBER OF ROWS": 1280, "OUTLIERS REMOVED": "0", "FIELDNAMES": "Date,AESO_Power_Demand,Calgary_Weather,Edmonton_Weather,FtMac_Weather", "COMPANY": "Alberta-Electric-System-Operator_AESO", "EMAIL": "support@otics.ca", "USERNAME": "admin", "IS NORMALIZED?": "Y", "PDF REPORT": "/maads/agentfilesdocker/dist/maadsweb/pdfreports/admin_aesopowerdemand_csv.pdf", "TRAINING FILE": "/maads/agentfilesdocker/dist/maadsweb/csvuploads/aesopowe

In [45]:
############ Hypertraining
#filename='aesopowerdemandlogistic.csv'
#dependentvariable='AESO_Power_Demand_Label'

filename='aesopowerdemand.csv'
#filename='aesopowerdemandsm.csv'
dependentvariable='AESO_Power_Demand'
removeoutliers=1
hasseasonality=1
deepanalysis=0

#hypertraining(host,port,filename,dependentvariable,removeoutliers,hasseasonality,deepanalysis)


Please wait...this could take 3-5 minutes
http://127.0.0.1:5595/?hypertraining=1&mode=0&username=admin&company=otics&email=support%40otics.ca&filename=aesopowerdemand.csv&removeoutliers=1&hasseasonality=0&dependentvariable=AESO_Power_Demand&summer=6%2C7%2C8&winter=11%2C12%2C1%2C2&shoulder=3%2C4%2C5%2C9%2C10&trainingpercentage=75&shuffle=1&deepanalysis=0
{"AlgoKey":"admin_aesopowerdemand_csv","AlgoDetails":"NuSVR,0.662062,allseason;GradientBoostingRegressor,0.651359,allseason;ADABoostRegressor,0.641686,allseason;DecisionTreeRegressor,0.613964,allseason;SVR,0.612398,allseason;RANSACRegressor,0.468207,allseason;LinearSVR,0.443493,allseason;HuberRegressor,0.441206,allseason;ARDRegression,0.439733,allseason;BayesianRidge,0.439699,allseason;","PDF":"/mnt/c/maads/agentfilesdocker/dist/maadsweb/pdfreports/admin_aesopowerdemand_csv_no_seasons.pdf","Hasseasonality":"No","Deep Analysis":"No","Shuffled":"Yes","Outliers Removed":"Yes","Generated On":"2024-04-08 21:00:07", "Timezone":"EDT","Username

In [ ]:
# ############Hyperpredictions
predictionport=5495
pkey='admin_aesopowerdemandlogistic_csv'
inputdata='6/10/2010,-14.3,-32.0,-12.0'
hyperprediction(pkey,host,predictionport,inputdata,'admin')

#hyperpredictioncustom(pkey,host,predictionport,inputdata,'admin','LogisticRegression','allseason')

pkey='admin_aesopowerdemand_csv'
inputdata='6/10/2010,-14.3,-52.0,-12.0'
hyperprediction(pkey,host,predictionport,inputdata,'admin')

algo='simpleregression_reg'
season='summer'
#hyperpredictioncustom(pkey,host,predictionport,inputdata,'admin',algo,season)
